In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

# Завантаження датасету CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Нормалізація зображень
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_test = (x_test.astype(np.float32) - 127.5) / 127.5

# Розміри зображень
img_shape = x_train.shape[1:]

# Розмір латентного простору
latent_dim = 100

# Створення генератора
def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(img_shape), activation='tanh'))
    model.add(layers.Reshape(img_shape))

    noise = tf.keras.Input(shape=(latent_dim,))
    img = model(noise)

    return tf.keras.Model(noise, img)

# Створення дискримінатора
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    img = tf.keras.Input(shape=img_shape)
    validity = model(img)

    return tf.keras.Model(img, validity)

# Компіляція моделей
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Замороження дискримінатора під час навчання генератора
discriminator.trainable = False

# Побудова GAN моделі
z = tf.keras.Input(shape=(latent_dim,))
img = generator(z)
valid = discriminator(img)

combined = tf.keras.Model(z, valid)
combined.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')

# Функція для навчання моделей
def train(epochs, batch_size=128, save_interval=50):
    # Завантаження та нормалізація даних
    (x_train, _), (_, _) = cifar10.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5

    # Етапи навчання
    for epoch in range(epochs):
        # Відбір випадкових зображень
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]

        # Генерація випадкового шуму та створення зображень
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Навчання дискримінатора
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Навчання генератора
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

        # Збереження прогресу
        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]}] [D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

    # Збереження моделей після навчання
    generator.save('generator.h5')
    discriminator.save('discriminator.h5')

# Навчання моделей
train(epochs=10000, batch_size=64, save_interval=1000)


Показано результат, скорочений до останніх рядків (5000).
2/2 [==============================] - 0s 19ms/step
2000 [D loss: 0.6551167368888855] [D accuracy: 69.53125%] [G loss: 2.4695053100585938]
2/2 [==============================] - 0s 29ms/step
3000 [D loss: 0.3807765245437622] [D accuracy: 83.59375%] [G loss: 2.7062721252441406]
2/2 [==============================] - 0s 26ms/step
4000 [D loss: 0.3043418824672699] [D accuracy: 88.28125%] [G loss: 3.3877487182617188]
2/2 [==============================] - 0s 22ms/step
5000 [D loss: 0.5549927353858948] [D accuracy: 69.53125%] [G loss: 2.9237568378448486]
2/2 [==============================] - 0s 24ms/step
6000 [D loss: 0.29013266414403915] [D accuracy: 87.5%] [G loss: 2.107177734375]
2/2 [==============================] - 0s 29ms/step
